## Packages Importing

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/sample_submission.csv
/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/train.csv
/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/test.csv
/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/cats


In [2]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict
from collections import Counter


from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)

In [58]:
import pickle

## Load Dataset

In [3]:
common_path = "/kaggle/input/fibe-hack-to-vibe-2-0-iab-dataset/{_fn}"

In [59]:
with open(common_path.format(_fn = "cats"), "rb") as fp:
    cats = pickle.load(fp)

In [18]:
df = pd.read_csv(common_path.format(_fn = "train.csv"))

In [19]:
df.shape

(697527, 4)

In [20]:
df.head(5)

,Unnamed: 0,text,target,Word Count
0,0,"python courses python courses, python exercise...",academic interests,125
1,1,the learning point open digital education. a r...,academic interests,147
2,2,"tech news, latest technology, mobiles, laptops...",academic interests,143
3,3,the best it certification materials in usa | k...,academic interests,364
4,4,"bioland scientific, for your research needs bi...",academic interests,176


In [21]:
# more cleaner way!
df = df \
.drop( "Unnamed: 0" , axis=1) \
.rename({"Word Count":"word_count"}, axis=1)

In [23]:
df.head(5)

,text,target,word_count
0,"python courses python courses, python exercise...",academic interests,125
1,the learning point open digital education. a r...,academic interests,147
2,"tech news, latest technology, mobiles, laptops...",academic interests,143
3,the best it certification materials in usa | k...,academic interests,364
4,"bioland scientific, for your research needs bi...",academic interests,176


In [34]:
cats_freq_df = df \
.groupby("target")["target"] \
.agg("count") 

In [36]:
cats_freq_df

target
academic interests                           58508
arts and culture                             26362
automotives                                  28004
books and literature                         40435
business and finance                         28038
careers                                      30484
family and relationships                     28951
food and drinks                              23434
health                                       18970
healthy living                               30945
hobbies and interests                        23955
home and garden                              23013
movies                                       21781
music and audio                              20630
news and politics                            30197
personal finance                             21132
pets                                         24136
pharmaceuticals, conditions, and symptoms    26596
real estate                                  23845
shopping                

In [38]:
fig = px.bar(cats_freq_df, x='target')
fig.show()

## Data Cleaning

In [39]:
# Special thanks to https://www.kaggle.com/tanulsingh077 for this function
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [42]:
def remove_stopwords(text):
    stop_words = stopwords.words('english')
    more_stopwords = ['u', 'im', 'c']
    stop_words = stop_words + more_stopwords
    
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text

In [41]:
def stemm_text(text):
    stemmer = nltk.SnowballStemmer("english")
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

In [43]:
def preprocess_test(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text = stemm_text(text)
    return text

In [44]:
df['text_clean'] = df.text.apply(preprocess_test)

In [47]:
df.head(5)

,text,target,word_count,text_clean
0,"python courses python courses, python exercise...",academic interests,125,python cours python cours python exercis pytho...
1,the learning point open digital education. a r...,academic interests,147,learn point open digit educ repositori tutori ...
2,"tech news, latest technology, mobiles, laptops...",academic interests,143,tech news latest technolog mobil laptop â nd...
3,the best it certification materials in usa | k...,academic interests,364,best certif materi usa killtest lead way stud...
4,"bioland scientific, for your research needs bi...",academic interests,176,bioland scientif research need bioland scienti...


In [50]:
df.to_csv("preprocessed_train_df_only_clean_text.csv", columns=["text_clean","target", "word_count"])

## Process the inference data

In [86]:
df_infer = pd.read_csv(common_path.format(_fn = "test.csv"))

In [87]:
df_infer['text_clean'] = df_infer.text.apply(preprocess_test)

In [90]:
df_infer.head(5)

,Unnamed: 0,text,Word Count,Index,text_clean
0,0,"equl offers enzyme assay kits, reagent mixture...",353,Article_0,equl offer enzym assay kit reagent mixtur enzy...
1,1,gauthmath: instant math questions solver for f...,112,Article_1,gauthmath instant math question solver free ac...
2,2,Whats the No. 1 cause of blindness in older ad...,340,Article_2,what caus blind older adult unit state scienc...
3,3,Surfers will ride a wave in the Amazon this we...,465,Article_3,surfer ride wave amazon week sciencethi week s...
4,4,"Why is the top of a leaf the most colorful, so...",269,Article_4,top leaf color sometim bright red undersid dra...


In [92]:
df_infer = df_infer \
.drop( "Unnamed: 0" , axis=1) \
.rename({"Word Count":"word_count"}, axis=1)

In [93]:
df_infer[ ["text_clean", "word_count", "Index"] ].head(5)

,text_clean,word_count,Index
0,equl offer enzym assay kit reagent mixtur enzy...,353,Article_0
1,gauthmath instant math question solver free ac...,112,Article_1
2,what caus blind older adult unit state scienc...,340,Article_2
3,surfer ride wave amazon week sciencethi week s...,465,Article_3
4,top leaf color sometim bright red undersid dra...,269,Article_4


In [94]:
df_infer.to_csv("preprocessed_infer_df_only_clean_text.csv", columns=["text_clean", "word_count", "Index"])

In [97]:
df_infer = df_infer[ ["text_clean", "word_count", "Index"] ]

## Label Encoding

In [46]:
from sklearn.preprocessing import LabelEncoder

In [51]:
le = LabelEncoder()
le.fit(df['target'])

LabelEncoder()

In [54]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [127]:
le_name_mapping

{'academic interests': 0,
 'arts and culture': 1,
 'automotives': 2,
 'books and literature': 3,
 'business and finance': 4,
 'careers': 5,
 'family and relationships': 6,
 'food and drinks': 7,
 'health': 8,
 'healthy living': 9,
 'hobbies and interests': 10,
 'home and garden': 11,
 'movies': 12,
 'music and audio': 13,
 'news and politics': 14,
 'personal finance': 15,
 'pets': 16,
 'pharmaceuticals, conditions, and symptoms': 17,
 'real estate': 18,
 'shopping': 19,
 'sports': 20,
 'style and fashion': 21,
 'technology and computing': 22,
 'television': 23,
 'travel': 24,
 'video gaming': 25}

In [128]:
inv_le_name_mapping = {}
for k,v in le_name_mapping.items():
    inv_le_name_mapping[v] = k

In [129]:
inv_le_name_mapping

{0: 'academic interests',
 1: 'arts and culture',
 2: 'automotives',
 3: 'books and literature',
 4: 'business and finance',
 5: 'careers',
 6: 'family and relationships',
 7: 'food and drinks',
 8: 'health',
 9: 'healthy living',
 10: 'hobbies and interests',
 11: 'home and garden',
 12: 'movies',
 13: 'music and audio',
 14: 'news and politics',
 15: 'personal finance',
 16: 'pets',
 17: 'pharmaceuticals, conditions, and symptoms',
 18: 'real estate',
 19: 'shopping',
 20: 'sports',
 21: 'style and fashion',
 22: 'technology and computing',
 23: 'television',
 24: 'travel',
 25: 'video gaming'}

In [60]:
cats

['academic interests',
 'books and literature',
 'healthy living',
 'careers',
 'news and politics',
 'shopping',
 'style and fashion',
 'family and relationships',
 'business and finance',
 'automotives',
 'pharmaceuticals, conditions, and symptoms',
 'arts and culture',
 'sports',
 'pets',
 'hobbies and interests',
 'real estate',
 'food and drinks',
 'home and garden',
 'video gaming',
 'movies',
 'travel',
 'personal finance',
 'technology and computing',
 'music and audio',
 'television',
 'health']

In [61]:
cats.sort()

In [62]:
cats

['academic interests',
 'arts and culture',
 'automotives',
 'books and literature',
 'business and finance',
 'careers',
 'family and relationships',
 'food and drinks',
 'health',
 'healthy living',
 'hobbies and interests',
 'home and garden',
 'movies',
 'music and audio',
 'news and politics',
 'personal finance',
 'pets',
 'pharmaceuticals, conditions, and symptoms',
 'real estate',
 'shopping',
 'sports',
 'style and fashion',
 'technology and computing',
 'television',
 'travel',
 'video gaming']

In [55]:
le_name_mapping

{'academic interests': 0,
 'arts and culture': 1,
 'automotives': 2,
 'books and literature': 3,
 'business and finance': 4,
 'careers': 5,
 'family and relationships': 6,
 'food and drinks': 7,
 'health': 8,
 'healthy living': 9,
 'hobbies and interests': 10,
 'home and garden': 11,
 'movies': 12,
 'music and audio': 13,
 'news and politics': 14,
 'personal finance': 15,
 'pets': 16,
 'pharmaceuticals, conditions, and symptoms': 17,
 'real estate': 18,
 'shopping': 19,
 'sports': 20,
 'style and fashion': 21,
 'technology and computing': 22,
 'television': 23,
 'travel': 24,
 'video gaming': 25}

In [63]:
for i in range(26):
    in_cats = cats[i]
    index_in_map = le_name_mapping[in_cats]
    if( i != index_in_map ):
        print(in_cats, i, index_in_map)

In [52]:
df['target_encoded'] = le.transform(df['target'])

In [64]:
df.head(5)

,text,target,word_count,text_clean,target_encoded
0,"python courses python courses, python exercise...",academic interests,125,python cours python cours python exercis pytho...,0
1,the learning point open digital education. a r...,academic interests,147,learn point open digit educ repositori tutori ...,0
2,"tech news, latest technology, mobiles, laptops...",academic interests,143,tech news latest technolog mobil laptop â nd...,0
3,the best it certification materials in usa | k...,academic interests,364,best certif materi usa killtest lead way stud...,0
4,"bioland scientific, for your research needs bi...",academic interests,176,bioland scientif research need bioland scienti...,0


## Vectorization

In [65]:
x = df['text_clean']
y = df['target_encoded']

print(len(x), len(y))

697527 697527


In [66]:
from sklearn.model_selection import train_test_split

In [76]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [77]:
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

627774 627774
69753 69753


In [78]:
from sklearn.feature_extraction.text import CountVectorizer

In [96]:
x_train.head(5)

412155    lockdown perfect time tollywood star introspec...
149617    varun tej lavanya tripathi hebah patel starrer...
205024    impact selfhelp pamphlet reduc risk drink amon...
169062    g thirda reddi pandem brought mix bag emot lea...
140679    intellectu capit suvarna samskriti darshik  na...
Name: text_clean, dtype: object

In [79]:
vect = CountVectorizer()
vect.fit(x_train)

CountVectorizer()

In [81]:
x_train_dtm = vect.transform(x_train)
x_test_dtm = vect.transform(x_test)

In [101]:
x_infer = df_infer['text_clean']

In [102]:
x_infer_dtm = vect.transform(x_infer)

## TF-IDF Transform

In [103]:
from sklearn.feature_extraction.text import TfidfTransformer

In [104]:
tfidf_transformer = TfidfTransformer()

In [105]:
tfidf_transformer.fit(x_train_dtm)

TfidfTransformer()

In [106]:
x_train_tfidf = tfidf_transformer.transform(x_train_dtm)

In [107]:
x_test_tfidf = tfidf_transformer.transform(x_test_dtm)

In [108]:
x_infer_tfidf = tfidf_transformer.transform(x_infer_dtm)

## Model NaiveBias

In [109]:
from sklearn.naive_bayes import MultinomialNB

In [110]:
nb = MultinomialNB()

In [112]:
nb.fit(x_train_dtm, y_train)

MultinomialNB()

In [113]:
y_pred_class = nb.predict(x_test_dtm)
y_pred_prob = nb.predict_proba(x_test_dtm)[:, 1]

In [115]:
from sklearn import metrics

In [120]:
metrics.f1_score(y_test, y_pred_class, average='macro')

0.6242399058037573

In [124]:
y_pred_class_infer = nb.predict(x_infer_dtm)

In [130]:
pd.Series(y_pred_class_infer, name="p_target_label")

0         17
1          0
2          9
3          9
4          7
          ..
174377    25
174378    25
174379    25
174380    25
174381    25
Name: p_target_label, Length: 174382, dtype: int64

In [131]:
pd.Series(y_pred_class_infer, name="p_target_label")\
.apply(lambda x : inv_le_name_mapping[x])

0         pharmaceuticals, conditions, and symptoms
1                                academic interests
2                                    healthy living
3                                    healthy living
4                                   food and drinks
                            ...                    
174377                                 video gaming
174378                                 video gaming
174379                                 video gaming
174380                                 video gaming
174381                                 video gaming
Name: p_target_label, Length: 174382, dtype: object

In [126]:
df_infer.head(5)

,text_clean,word_count,Index
0,equl offer enzym assay kit reagent mixtur enzy...,353,Article_0
1,gauthmath instant math question solver free ac...,112,Article_1
2,what caus blind older adult unit state scienc...,340,Article_2
3,surfer ride wave amazon week sciencethi week s...,465,Article_3
4,top leaf color sometim bright red undersid dra...,269,Article_4


In [133]:
sub_df = df_infer[ ['Index'] ]

In [134]:
sub_df['target'] = pd.Series(y_pred_class_infer, name="p_target_label")\
.apply(lambda x : inv_le_name_mapping[x])

In [135]:
sub_df.head(5)

,Index,target
0,Article_0,"pharmaceuticals, conditions, and symptoms"
1,Article_1,academic interests
2,Article_2,healthy living
3,Article_3,healthy living
4,Article_4,food and drinks


In [136]:
sub_df.to_csv("nb.csv", columns=["Index","target"], index=False)

## XGBoost

In [138]:
import xgboost as xgb

In [139]:
xbg_clf = xgb.XGBClassifier(
        learning_rate=0.1,
        max_depth=7,
        n_estimators=80,
        use_label_encoder=False,
        eval_metric='auc',
        # colsample_bytree=0.8,
        # subsample=0.7,
        # min_child_weight=5,
    )

In [ ]:
xbg_clf.fit(x_train_dtm, y_train)

In [1]:
y_pred_class = xbg_clf.predict(x_test_dtm)

NameError: name 'xbg_clf' is not defined